## Investigate model architectures to broadcast events from andoe to gate

Strategy:
- Could exploit expected homogenity -> But until what radius -> The edges are massively underpopulated for expectation -> Plot r^2 vs event dist normalized to relative observed area per bin, take cutoff when distribution drops -> Poor edge performance
- Use Background edge events? 

Also need to find dataset where i can resolve the anode mesh 

Could also try using the naive anode model

Or use Ripley Statistic for homogeneity train NN inspect functional form at different location in tpc, most important will be walls, and around anode/gate perp and mesh and find some function that can fit all three of these

Ripley functions:
- K : The sample-based estimate is defined as:
$$\hat{K}(t) = \lambda^{-1}\sum_{i\neq j}\frac{I(d_{ij}<t)}{n}$$
where d is the euclidean distance, t is the search radius, $\lambda$ is the average point density (over the entire set) and I is the indicator function (1 if true, 0 if false : Ie its a boolean)

If points are homogeneous we expect that $\hat{K}(t)=\pi t^2$ for 2D data. 

- L : The variance stabilized Ripley K function, the sample based function is:
$$\hat{L}(t) = \left(\frac{\hat{K}(t)}{\pi}\right)^{1/2}$$

For a homogeneous distribution we expect that $\hat{L}(t) = t$ with variance in t being approximately constant (ie the same for all t)



Also note: Fucking piece of shit tensorflow cant cast values so I cant use Truth values for mathematics without breaking differentiability

In [1]:
import straxen

import sys, os
import h5py

import tensorflow
from tensorflow import keras
import numpy as jnp

sys.path.append(os.path.join(os.getcwd(), 'ActualThesisWork'))
%load_ext autoreload
%autoreload 2
from functions import *


tensorflow.keras.backend.set_floatx('float32')

2024-11-29 12:07:16,811 - admix - WARNING - Initializing utilix DB failed. You cannot do database operations


DB initialization failed


2024-11-29 12:07:17.591095: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 12:07:17.592843: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 12:07:17.613350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 12:07:17.613373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 12:07:17.614100: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:

# Basic variables
dead_pmts = jnp.array([ 28, 108, 121, 144, 156, 164, 177])
not_dead_pmts = jnp.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252])
pmt_radius = straxen.tpc_pmt_radius# 3  * 2.54 / 2 
tpc_radius = 66.4
max_z = straxen.tpc_z
I0_init_from_ULCE = 0.1 # This is never act used
pmt_pos = straxen.pmt_positions()
pmt_pos_top = pmt_pos[pmt_pos.array == "top"].to_numpy()
n_pmts = pmt_pos_top.shape[0]

lambda_ = jnp.pi * tpc_radius**2

def RipleyK(dat, t):
    pairwise_dist = jnp.linalg.norm(dat[:, jnp.newaxis, :] - dat[np.newaxis, :, :], axis=2)
    indicator_func = (pairwise_dist < t)
    jnp.fill_diagonal(indicator_func, 0) # i \neq j 
    return (1/(lambda_ * dat.shape[0])) * jnp.sum(indicator_func)

def RipleyL(dat, tpc_radius, min_t, samples=100):
    ts = jnp.linspace(min_t, tpc_radius, samples)
    Ls = jnp.zeros_like(ts)
    for i in range(len(Ls)):
        Ls[i] = jnp.sqrt(RipleyK(dat, ts[i])/np.pi)
    return Ls, ts

# And tensorflow version
def RipleyK_vectorized(dat, ts):
    """
    Compute Ripley's K for multiple t values in a vectorized way.
    """
    # Compute pairwise distances
    dat_expanded_1 = tensorflow.expand_dims(dat, axis=1)  # Shape: (N, 1, D)
    dat_expanded_2 = tensorflow.expand_dims(dat, axis=0)  # Shape: (1, N, D)
    pairwise_dist = tensorflow.norm(dat_expanded_1 - dat_expanded_2, axis=2)  # Shape: (N, N)
    
    # Expand ts for comparison (N, N, samples)
    ts_expanded = tensorflow.reshape(ts, (1, 1, -1))
    
    # Compute indicator function for each t
    #indicator_func = tensorflow.cast(pairwise_dist[:, :, tensorflow.newaxis] < ts_expanded, tensorflow.float32)
    
    # Remove diagonal (i != j)
    # Mask diagonal elements for all samples
    #diag_mask = tensorflow.eye(tensorflow.shape(pairwise_dist)[0], dtype=tensorflow.float32)  # Shape: (N, N)
    #diag_mask = tensorflow.expand_dims(diag_mask, axis=-1)  # Shape: (N, N, 1)
    #indicator_func *= (1 - diag_mask)  # Mask diagonal by multiplying

    beta = 100.0  # Controls sharpness of the transition
    indicator_func = 1.0 / (1.0 + tensorflow.exp(-beta * (ts_expanded - pairwise_dist[:, :, tensorflow.newaxis])))
    
    # Sum over all pairs and normalize
    #N = tensorflow.cast(tensorflow.shape(dat)[0], tensorflow.float32)  # Number of points
    RipleyK_values = (1 / (lambda_ )) * tensorflow.reduce_sum(indicator_func, axis=[0, 1])  # Shape: (samples,)
    
    return RipleyK_values

def RipleyL_vectorized(dat, tpc_radius, min_t, samples=100):
    """
    Compute Ripley's L in a fully vectorized way.
    """
    # Generate range of t values
    ts = tensorflow.linspace(min_t, tpc_radius, samples)  # Shape: (samples,)
    
    # Compute Ripley's K for all t values
    K_values = RipleyK_vectorized(dat, ts)
    
    # Transform to Ripley's L
    L_values = tensorflow.sqrt(K_values / tensorflow.constant(np.pi, dtype=tensorflow.float32))
    
    return L_values, ts

### Loss function

We want to penalize inhomogeneity (assuming events come from everywhere)

Penalize large movements  > 5mm maybe > 2.5mm given the symmetry

Prob pretrain on MC? 


In [3]:
def loss_func_gen(t, t_min=0.1, t_max=tpc_radius, samples=10, penalize_shift=1.0, max_shift=0.5):
    def loss_func(pos_true, pos_pred):
        #Rl, ts = RipleyL_vectorized(pos_pred, t_max, t_min, samples)
        #loss = tensorflow.square(Rl - ts)
        loss = RipleyK_vectorized(pos_pred*tpc_radius, t)
        #loss += tensorflow.where(
        #        tensorflow.greater(
        #            tensorflow.math.abs(tensorflow.math.reduce_euclidean_norm(pos_true, 1, keepdims=True)-tensorflow.math.reduce_euclidean_norm(pos_pred, 1, keepdims=True)), 
        #            max_shift), 
        #        penalize_shift, 0.0)
        return loss
    return loss_func

In [4]:
import math
def anode_mesh_perp_wire_dist(positions, tpc_radius):
    """Taken from Shenyang https://github.com/XENONnT/analysiscode/blob/master/event_patternfit/neural_net_patternfit/Neural_tuned_mc_kr.ipynb"""
    angle = -math.pi/3
    xx_rotated = tensorflow.expand_dims((positions[:,0]*tensorflow.math.cos(angle) - positions[:,1]*tensorflow.math.sin(angle)),axis=-1)
    yy_rotated = tensorflow.expand_dims((positions[:,0]*tensorflow.math.sin(angle) + positions[:,1]*tensorflow.math.cos(angle)),axis=-1)
    dist_anode_mesh = tensorflow.math.minimum(xx_rotated - (xx_rotated//0.5)*0.5, 0.5 - (xx_rotated - (xx_rotated//0.5)*0.5) ) / tpc_radius
    # Now the perpendicular wires, in order we have gate0,gate1,anode0,anode1,... this is in rotated frame
    wire_pos = tensorflow.constant([[-263, 263, -318, -283, 283, 318]], dtype=tensorflow.float32)
    dist_perp = xx_rotated - wire_pos
    dist_wall = (tpc_radius - tensorflow.math.reduce_euclidean_norm(positions, 1, keepdims=True) )
    
    dist_anode_mesh /= 0.5
    dist_perp /= tpc_radius
    dist_wall /= tpc_radius
    positions /= tpc_radius
    # return n,10 from n,6 n,1 n,1 and n,2
    return tensorflow.concat([dist_perp, dist_anode_mesh, dist_wall, positions], axis=1)

class GenerateCoordinates(keras.layers.Layer):
    def __init__(self,tpc_radius):
        self.tpc_radius = tpc_radius
        super().__init__()

    def call(self, X):
        return anode_mesh_perp_wire_dist(X, self.tpc_radius)

## Get Data

We need data formated for anode positions, so we apply the rLCE

In [5]:
# Base data 
import h5py
def get_filtered(path):
    with h5py.File(path, 'r') as f:
        patterns = f['patterns'][:]
        ei = f['event_info'][:]
        # run_ids = f["run_ids"][:]
        print("Total Livetime {}s".format(f.attrs["Livetime_tot_s"]))
        print("Total Datapoints pre filter {}".format(f.attrs["OriginalSize"]))
        print("Total Datapoints post filter {}".format(f.attrs["CurrentSize"]))
    assert len(patterns) == len(ei)
    return patterns, ei

filtered_path = '/Code/processed_data/SrCut_z20_0000.hdf5'

patterns, ei = get_filtered(filtered_path)
positions = jnp.stack([ei['s2_x'], ei['s2_y']],axis = -1)
patterns = patterns['s2_area_per_channel'][:, :n_pmts]
patterns = tensorflow.convert_to_tensor(patterns, dtype=tensorflow.float32)
positions = tensorflow.convert_to_tensor(positions, dtype=tensorflow.float32)

Total Livetime 235843.858s
Total Datapoints pre filter 1314844
Total Datapoints post filter 184657


In [6]:
# Refine Data
allow_njit = True
keras_loss = logl_loss_generator(pmt_pos_top, keras.backend, not_dead_pmts = not_dead_pmts, cap=float('inf'), allow_njit=allow_njit)
if allow_njit:
    keras_loss(np.random.random((1, n_pmts)),np.random.random((1, n_pmts)))

def get_minimal_LCE_coordinate_model(n_pmts, pmt_positions, n_pmt_pos = 1):
    """
    Generate's PMT Position coordinate, the main one used for now
    """
    conv_pmt_coords = keras.models.Sequential([
        keras.layers.RepeatVector(n_pmts, input_shape=(2,)),
        GetRadius((n_pmts, n_pmt_pos), keras.backend.variable(pmt_positions)),
    ], name='xy_to_pmt_coords')
    return conv_pmt_coords

def get_RLCEModel(pmt_pos_top,n_pmts, n_groups, group_slices, I0_init, guess,**kwargs):
    """
    Generate Radial LCE Model for fitting
    kwargs -> Overflow from old variables
    """
    # Input layer
    input_pos = keras.layers.Input(shape=(2))
    # Get different coordinates
    rho = get_minimal_LCE_coordinate_model(n_pmts = n_pmts, pmt_positions = pmt_pos_top, n_pmt_pos = 1)(input_pos)
    # Apply radial lce 
    m = RadialLCELayer(name='radial_lce',guess= guess, n_groups=n_groups, group_slices=group_slices)(rho)
    
    # Holds I0 for each PMT
    m = I0Layer(n_pmts, init_val= tensorflow.cast(I0_init, tensorflow.float32), name='QE_Layer')(m)
    # Normalization Layer 
    m = NormalizationLayer(name='Normalization')(m)
    return keras.models.Model(inputs=input_pos, outputs=m, name="Radial_LCE")

npmt_pd = jnp.array([ 1.6263583e+00,  9.4909382e+00, -4.1542644e-05,  7.7986442e-03])
I0s = jnp.array([0.004235879983752966, 0.004732195753604174, 0.00466069346293807, 0.004735287744551897, 0.004843718837946653, 0.0036560967564582825, 0.004791035782545805, 0.004351622425019741, 0.004277762491255999, 0.003982590511441231, 0.0036782578099519014, 0.004139808006584644, 0.003963056020438671, 0.004514740779995918, 0.0048719532787799835, 0.003294900059700012, 0.004271252546459436, 0.004213831853121519, 0.003675800282508135, 0.003923185635358095, 0.004053255543112755, 0.003687959862872958, 0.003949134610593319, 0.0039024786092340946, 0.003996435087174177, 0.0045274775475263596, 0.00314155500382185, 0.004692835733294487, 0.0017648281063884497, 0.003712652949616313, 0.0041131614707410336, 0.0039019035175442696, 0.0038532845210283995, 0.003392500802874565, 0.003896859474480152, 0.003918000962585211, 0.004090684931725264, 0.004209689795970917, 0.004247184377163649, 0.0050539360381662846, 0.004631971474736929, 0.0039286985993385315, 0.003714651335030794, 0.003667386481538415, 0.004035579971969128, 0.003805769607424736, 0.003694107523187995, 0.0038736388087272644, 0.004259074572473764, 0.0036939873825758696, 0.004129675682634115, 0.004015707410871983, 0.0042891716584563255, 0.005394953768700361, 0.0049893721006810665, 0.004004128277301788, 0.0036364563275128603, 0.0037317327223718166, 0.0037540518678724766, 0.004226438235491514, 0.003232576884329319, 0.004007598850876093, 0.003992777783423662, 0.00392210902646184, 0.003881508018821478, 0.0037358906120061874, 0.0034426788333803415, 0.004593325313180685, 0.004476095549762249, 0.004509852733463049, 0.00398320984095335, 0.0038602910935878754, 0.0038099722005426884, 0.0035987216979265213, 0.004056497476994991, 0.003779412480071187, 0.003909052349627018, 0.003880080534145236, 0.0037605699617415667, 0.0040501500479876995, 0.003605334088206291, 0.0037680561654269695, 0.003598541719838977, 0.0036737960763275623, 0.004982828162610531, 0.004214885178953409, 0.00394063163548708, 0.00341225927695632, 0.0037580307107418776, 0.003971084486693144, 0.004098842386156321, 0.003986097406595945, 0.004088491201400757, 0.004092586226761341, 0.003926937002688646, 0.004298168700188398, 0.004080779384821653, 0.003439064836129546, 0.003919169306755066, 0.0038085675332695246, 0.004447304643690586, 0.004079096484929323, 0.004161607474088669, 0.0035417259205132723, 0.004211203660815954, 0.003966113086789846, 0.004028564319014549, 0.0038740618620067835, 0.001677889609709382, 0.004096835386008024, 0.004131924360990524, 0.004420299082994461, 0.0038429093547165394, 0.003977605141699314, 0.004036216530948877, 0.0040572527796030045, 0.0034212754108011723, 0.004304381553083658, 0.004800576251000166, 0.003856675000861287, 0.0037602523807436228, 0.0017064257990568876, 0.003873104928061366, 0.003873534733429551, 0.003949661739170551, 0.004201176576316357, 0.0039010290056467056, 0.003948729019612074, 0.004660475999116898, 0.003974431660026312, 0.004105430096387863, 0.003848433494567871, 0.0039720600470900536, 0.004035878926515579, 0.005145389586687088, 0.004992862232029438, 0.004138431046158075, 0.0038946077693253756, 0.0037270355969667435, 0.003934138920158148, 0.004055908881127834, 0.004122219048440456, 0.00413487758487463, 0.004022635985165834, 0.0016843285411596298, 0.0039187888614833355, 0.004069270566105843, 0.004088082350790501, 0.0038323558401316404, 0.003933216445147991, 0.0041273306123912334, 0.00391603447496891, 0.003779760794714093, 0.0038304287008941174, 0.003971415106207132, 0.003770100185647607, 0.0016588973812758923, 0.003752924269065261, 0.0038747305516153574, 0.004287980031222105, 0.004396375268697739, 0.004206789191812277, 0.004279491025954485, 0.0038022841326892376, 0.0016486833337694407, 0.003921540919691324, 0.00393262505531311, 0.0036610763054341078, 0.005085785873234272, 0.00415172940120101, 0.0035354148130863905, 0.004259951412677765, 0.0040181828662753105, 0.003962669055908918, 0.004295114893466234, 0.004105494357645512, 0.00389536889269948, 0.0016863005002960563, 0.004174431320279837, 0.0035444151144474745, 0.0036017769016325474, 0.0038141191471368074, 0.0036408151499927044, 0.004543586168438196, 0.004658541176468134, 0.0037312631029635668, 0.0039724307134747505, 0.0036708605475723743, 0.003768590744584799, 0.003977857064455748, 0.003958065528422594, 0.00391046516597271, 0.004013632424175739, 0.004107540939003229, 0.004054495599120855, 0.003917690832167864, 0.0037647797726094723, 0.003846473526209593, 0.005038938019424677, 0.005282974801957607, 0.003871278138831258, 0.004100256133824587, 0.003908885642886162, 0.004233073443174362, 0.003930484410375357, 0.0037340044509619474, 0.0039030059706419706, 0.0037919552996754646, 0.00414620153605938, 0.003158329986035824, 0.0034461934119462967, 0.004152875859290361, 0.005023165140300989, 0.005013687536120415, 0.004022886045277119, 0.0038592142518609762, 0.003895816160365939, 0.004128684755414724, 0.0037005161866545677, 0.00402730843052268, 0.003608384868130088, 0.0027551278471946716, 0.004201559349894524, 0.0035797676537185907, 0.003656105836853385, 0.004713757894933224, 0.003788005094975233, 0.004254915285855532, 0.0041169957257807255, 0.004034965764731169, 0.0036038379184901714, 0.003389847930520773, 0.003905089572072029, 0.0038801683112978935, 0.0035936341155320406, 0.004151667468249798, 0.0040817963890731335, 0.004347157198935747, 0.004485994577407837, 0.00449867220595479, 0.004051011521369219, 0.0041497559286653996, 0.00477506872266531, 0.004078592173755169, 0.004272844642400742, 0.0041829245164990425, 0.005090698599815369, 0.003888470819219947, 0.004583718255162239, 0.005087660159915686, 0.0047324057668447495, 0.0045408145524561405, 0.0037471496034413576])

radial_lce = get_RLCEModel(pmt_pos_top[:,2:],n_pmts=n_pmts, n_groups=1, group_slices=np.arange(n_pmts), 
                        I0_init=I0_init_from_ULCE, guess = npmt_pd.tolist())
radial_lce.layers[-2].set_weights([I0s])

def get_position_updater(positions, n_to_update, model,input_len = 1, embeddings_constraint=[]):
    """
    Returns encoder model to get patterns from positions
    """
    input = keras.layers.Input(shape=(input_len,))
    if embeddings_constraint != []:
        m = keras.layers.Embedding(n_to_update, 2, input_length = input_len, name='embed_pos',embeddings_constraint=embeddings_constraint)(input)
    else: 
        m = keras.layers.Embedding(n_to_update, 2, input_length = input_len, name='embed_pos')(input)
    m = keras.layers.Flatten()(m)
    m = model(m)
    pos_updater = keras.models.Model(inputs= input, outputs=m, name="Position_Updater")
    pos_updater.layers[-1].trainable = False
    pos_updater.get_layer('embed_pos').set_weights([positions[:n_to_update]])

    return pos_updater
def train_and_update_pos(positions, patterns, model, n_iter = 5, verbose=0, hist_pos=False, keras_loss=keras_loss):
    early_stopping = tensorflow.keras.callbacks.EarlyStopping(
        monitor='loss',
        restore_best_weights=True,
        patience=4,    
        min_delta=0.05,
        mode='min',    
        verbose=1      
    )
    reduce_lr = tensorflow.keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        mode = 'min',
        factor=0.5,     
        patience=2,     
        min_delta=0.05, 
        min_lr=1e-3,    
        verbose=1       
    )
    n_tot = len(positions)
    pos_updater = get_position_updater(positions, n_tot, model ,input_len = 1)
    # Inspecting the degree of change
    if hist_pos:
        trained_pos = []

    new_pos = tensorflow.identity(positions)
    loss    = keras_loss(patterns, model.predict(positions))
    improved= jnp.zeros(positions.shape[0])
    improved[:] = -1
    for i in range(n_iter):
        print("Jagged iter {}".format(i))
        # tensorflow_probability requires tensorflow >= 2.16 we have == 2.15
        # Random dist gives max deviation root{2}*pmt_radius (~70% of diameter)
        noise_pos = tensorflow.identity(positions) + jnp.random.rand(*new_pos.numpy().shape) * pmt_radius
        # Update and fit encoder model positions
        pos_updater.get_layer('embed_pos').set_weights([noise_pos])
        # Compile and fit
        pos_updater.compile(loss= keras_loss, optimizer = keras.optimizers.Adam(learning_rate=1e-1))
        pos_updater.fit(x = jnp.arange(n_tot), y = patterns, 
                        epochs = 20, batch_size = 2**12,
                        callbacks=[early_stopping,reduce_lr], verbose = verbose) 
        noise_pos = pos_updater.get_layer('embed_pos').get_weights()[0]
        # Check result of new pos
        jittered_loss = keras_loss(patterns, model.predict(noise_pos, verbose=verbose))

        better = jittered_loss < loss
        loss =    tensorflow.where(better, jittered_loss, loss) 
        new_pos = tensorflow.where(tensorflow.expand_dims(better, axis=-1), noise_pos, new_pos) 
        improved[better.numpy()] = i
        if hist_pos:
            trained_pos.append(noise_pos)

    print("Jittering Results")
    print("Didnt improve : {}%".format((improved == -1).mean()*100))
    for i in range(n_iter):
        print("Iteration {} : {}%".format(i, (improved == i).mean()*100))
    if not hist_pos:
        return new_pos
    else:
        return new_pos, trained_pos
        
new_pos=train_and_update_pos(positions, patterns, radial_lce, n_iter = 10, verbose=1, hist_pos=False).numpy()

5771/5771 [==============================] - 5s 900us/step
Jagged iter 0
Epoch 1/20
46/46 [==============================] - 1s 11ms/step - loss: 3.6173 - lr: 0.1000
Epoch 2/20
46/46 [==============================] - 0s 11ms/step - loss: 3.0383 - lr: 0.1000
Epoch 3/20
46/46 [==============================] - 1s 11ms/step - loss: 2.6332 - lr: 0.1000
Epoch 4/20
46/46 [==============================] - 0s 11ms/step - loss: 2.3766 - lr: 0.1000
Epoch 5/20
46/46 [==============================] - 0s 10ms/step - loss: 2.2260 - lr: 0.1000
Epoch 6/20
46/46 [==============================] - 0s 11ms/step - loss: 2.1434 - lr: 0.1000
Epoch 7/20
46/46 [==============================] - 0s 10ms/step - loss: 2.1008 - lr: 0.1000
Epoch 8/20
46/46 [==============================] - 1s 11ms/step - loss: 2.0801 - lr: 0.1000
Epoch 9/20
46/46 [==============================] - 0s 11ms/step - loss: 2.0708 - lr: 0.1000
Epoch 10/20
41/46 [=========================>....] - ETA: 0s - loss: 2.0666
Epoch 10: Redu

### Back to Anode to Gate Model

In [143]:
"""
Multi Layer MLP


Using tutorial code as starting point
"""
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

n_in = 2

# ------------------------------- Init Weights ---------------------------------
def random_layer_params(m, n, key, scale):
    # Random init of weights
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n,m)), scale*random.normal(b_key, (n,))

def init_network_params(sizes, key, scale):
    keys = random.split(key,len(sizes))
    return [random_layer_params(m,n,k, scale) for m,n,k in zip(sizes[:-1],sizes[1:],keys)]

# ------------------------------- Training func --------------------------------
from jax.scipy.special import logsumexp

def relu(x): # TODO Change
  return jnp.maximum(0, x)

def predict(params, image):
    activations = image

    # For weight, bias in layers
    for w,b in params[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = relu(outputs)
    # Get last layer
    final_w, final_b = params[-1]
    return jnp.dot(final_w, activations) + final_b

batched_predict = vmap(predict, in_axes=(None, 0))


In [148]:
# Model params

layer_sizes = [2, 2**8, 2**12, 2**8, 2]
step_size = 0.01
num_epochs = 10
batch_size = 128
n_targets = 10
params = init_network_params(layer_sizes, random.key(0),9e-2)

In [144]:
# Def function to actually compute per item loss
from jax import lax
def indicator_function_lazy(dat, t):
    n = dat.shape[0]
    def compute_row(i, acc):
        row = lax.dynamic_index_in_dim(dat, i, axis=0, keepdims=False) 
        distances = jnp.linalg.norm(row - dat, axis=1)
        #indicator_row = (distances < t).at[i].set(0.0)  # Set diagonal to 0
        indicator_row = (1 / (1 + jnp.exp(100 * (distances - t)))).at[i].set(0.0)
        return acc + jnp.sum(indicator_row)
    result = lax.fori_loop(0, n, compute_row, 0.0)
    return result
@jit
def RipleyK(dat, t, lambda_):
    #pairwise_dist = jnp.linalg.norm(dat[:, jnp.newaxis, :] - dat[jnp.newaxis, :, :], axis=2)
    #diagonal_mask = ~jnp.eye(pairwise_dist.shape[0], dtype=bool)  # Mask for i != j
    #indicator_func = (pairwise_dist < t) * diagonal_mask
    return (1/(lambda_ * dat.shape[0])) * indicator_function_lazy(dat, t)

def RipleyL(dat, tpc_radius, min_t, lambda_,samples=10, ):
    ts = jnp.linspace(min_t, tpc_radius, samples)
    RipleyK_vmap = jax.vmap(lambda t: jnp.sqrt(RipleyK(dat, t, lambda_) / jnp.pi))
    Ls = RipleyK_vmap(ts)
    return Ls, ts
@jit
def accuracy(params, x_pos, lambda_,min_t, tpc_radius, y_pos = None):
    return RipleyL(batched_predict(params, x_pos), tpc_radius=tpc_radius, min_t=min_t,lambda_ = lambda_, samples = 4) # 1 mm -> 5mm grid spacing

# Def loss function -> Returns single item
@jit
def loss(params, x_pos,lambda_,t, y_pos = None):
    tmp = RipleyK(batched_predict(params, x_pos), t=t, lambda_ = lambda_)
    #print(tmp)
    return tmp

# Def function to update weights
@jit
def update(params, x, y,  lambda_, step_size, t):
    grads = grad(loss)(params, x, lambda_, t)
    #print(grads)
    return [(w - step_size * dw, b - step_size * db)
            for (w, b), (dw, db) in zip(params, grads)]

def create_mini_batches(x, y, batch_size, key):
    num_samples = len(x)
    key, subkey = jax.random.split(key)
    indices = jax.random.permutation(subkey, num_samples)
    x_shuffled = x[indices]
    y_shuffled = y[indices]
    
    for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)
        yield x_shuffled[start:end], y_shuffled[start:end]

In [145]:
# Get indices
shuffle = np.random.permutation(len(new_pos))
train_indcs, test__indcs = shuffle[0: int(len(new_pos) *0.8)], shuffle[int(len(new_pos) *0.8): len(new_pos)]
unscale = (new_pos.mean(), new_pos.std())
new_pos_scale = (new_pos - unscale[0]) / unscale[1]
x_train, y_train = new_pos_scale[train_indcs], new_pos_scale[train_indcs]
x_test,  y_test  = new_pos_scale[test__indcs], new_pos_scale[test__indcs]

In [146]:
# TODO Peanalize large shifts in loss
key = jax.random.PRNGKey(42)

step_size = 0.1


for epoch in range(num_epochs):
    start_time = time.time()
    # Iterate batches and do thing
    key, subkey = jax.random.split(key)
    for x, y in create_mini_batches(x_train, y_train, batch_size, subkey):
        # This applies loss func and updates weights
        params = update(params, x, y, lambda_=jnp.pi * (tpc_radius/unscale[1])**2, step_size=step_size, t = 2/unscale[1])
    epoch_time = time.time() - start_time

    print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
    #Ls, ts = accuracy(params, x_train)
    #print("Training set Ripley L var: {}; mean: {}".format(jnp.std(Ls-ts), jnp.mean(Ls-ts)))
    Ls, ts = accuracy(params, x_test, lambda_=jnp.pi * (tpc_radius/unscale[1])**2, min_t = (0.1/unscale[1]), tpc_radius=tpc_radius/unscale[1])
    print("Test set Ripley L var: {}; mean: {}".format(jnp.std(Ls-ts), jnp.mean(Ls-ts)))
    #Ls, ts = accuracy(params, new_pos)
    #print("Full set Ripley L var: {}; mean: {}".format(jnp.std(Ls-ts), jnp.mean(Ls-ts)))
    #gc.collect()


Epoch 0 in 9.47 sec
Test set Ripley L var: nan; mean: nan


KeyboardInterrupt: 

In [156]:
grads = grad(loss)(params, x, jnp.pi * (tpc_radius/unscale[1])**2, t=1)
print(f"Gradients: {grads}")

Gradients: [(Array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],

In [167]:
grads[0][0].shape, grads[0][1].shape, grads[1][0].shape, grads[1][1].shape

((256, 2), (256,), (4096, 256), (4096,))

### Learning JAX - Above rewritten version of below

In [ ]:
# Really basic model to understand what im doing
"""
Single Layer MLP


key corresponds to random initializer for reproducibility, random.split(key) produces subkeys

"""
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random


# ------------------------------- Init Weights ---------------------------------
def random_layer_params(m, n, key, scale=1e-2):
    # Random init of weights
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n,m)), scale*random.normal(b_key, (n,))

def init_network_params(sizes, key):
    keys = random.split(key,len(sizes))
    return [random_layer_params(m,n,k) for m,n,k in zip(sizes[:-1],sizes[1:],keys)]

# ------------------------------- Training func --------------------------------
from jax.scipy.special import logsumexp

def relu(x):
  return jnp.maximum(0, x)

def predict(params, image):
    activations = image

    # For weight, bias in layers
    for w,b in params[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = relu(outputs)
    # Get last layer
    final_w, final_b = params[-1]
    # Logits are raw unnormalized output params before activation function 
    logits = jnp.dot(final_w, activations) + final_b
    return logits - logsumexp(logits)

In [ ]:
# Model params

layer_sizes = [784, 512, 512, 10]
step_size = 0.01
num_epochs = 10
batch_size = 128
n_targets = 10
params = init_network_params(layer_sizes, random.key(0))

In [ ]:
random_flattened_image = random.normal(random.key(1), (28 * 28,))
preds = predict(params, random_flattened_image)
print(preds.shape)

(10,)


In [ ]:
# Apply predict that works on single item to batches
batched_predict = vmap(predict, in_axes=(None, 0))

random_flattened_images = random.normal(random.key(1), (10, 28 * 28))
batched_preds = batched_predict(params, random_flattened_images)
print(batched_preds.shape)

(10, 10)


In [ ]:
def one_hot(x, k, dtype=jnp.float32):
    """Create a one-hot encoding of x of size k."""
    return jnp.array(x[:, None] == jnp.arange(k), dtype)

# Def function to actually compute per item loss
def accuracy(params, images, targets):
    target_class = jnp.argmax(targets, axis=1)
    predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
    return jnp.mean(predicted_class == target_class)

# Def loss function -> Returns single item
def loss(params, images, targets):
    preds = batched_predict(params, images)
    return -jnp.mean(preds * targets)

# Def function to update weights
@jit
def update(params, x, y):
    grads = grad(loss)(params, x, y)
    return [(w - step_size * dw, b - step_size * db)
            for (w, b), (dw, db) in zip(params, grads)]


In [ ]:
%pip install tensorflow_datasets

/opt/XENONnT/anaconda/envs/XENONnT_el9.2024.10.4/lib/python3.9/pty.py:85: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.0 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21483 sha256=80c670f1542e77d8c14943449d0aa18a8861e2a1b7cedb937dd8dc1bd0810572
  Stored in directory: /root/.cache/pip/wheels/e1/e8/83/ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0.post1 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.5.0 which is incompatible.
Not

In [ ]:
# Dataload with tf for images
import tensorflow as tf
# Ensure TF does not see GPU and grab all GPU memory.
tf.config.set_visible_devices([], device_type='GPU')

import tensorflow_datasets as tfds

data_dir = '/tmp/tfds'

# Fetch full datasets for evaluation
# tfds.load returns tf.Tensors (or tf.data.Datasets if batch_size != -1)
# You can convert them to NumPy arrays (or iterables of NumPy arrays) with tfds.dataset_as_numpy
mnist_data, info = tfds.load(name="mnist", batch_size=-1, data_dir=data_dir, with_info=True)
mnist_data = tfds.as_numpy(mnist_data)
train_data, test_data = mnist_data['train'], mnist_data['test']
num_labels = info.features['label'].num_classes
h, w, c = info.features['image'].shape
num_pixels = h * w * c

# Full train set
train_images, train_labels = train_data['image'], train_data['label']
train_images = jnp.reshape(train_images, (len(train_images), num_pixels))
train_labels = one_hot(train_labels, num_labels)

# Full test set
test_images, test_labels = test_data['image'], test_data['label']
test_images = jnp.reshape(test_images, (len(test_images), num_pixels))
test_labels = one_hot(test_labels, num_labels)

print('Train:', train_images.shape, train_labels.shape)
print('Test:', test_images.shape, test_labels.shape)


Train: (60000, 784) (60000, 10)
Test: (10000, 784) (10000, 10)


In [ ]:
import time

# Batch training data
def get_train_batches():
    # as_supervised=True gives us the (image, label) as a tuple instead of a dict
    ds = tfds.load(name='mnist', split='train', as_supervised=True, data_dir=data_dir)
    # You can build up an arbitrary tf.data input pipeline
    ds = ds.batch(batch_size).prefetch(1)
    # tfds.dataset_as_numpy converts the tf.data.Dataset into an iterable of NumPy arrays
    return tfds.as_numpy(ds)

for epoch in range(num_epochs):
    start_time = time.time()
    # Iterate batches and do thing
    for x, y in get_train_batches():
        x = jnp.reshape(x, (len(x), num_pixels))
        y = one_hot(y, num_labels)
        # This applies loss func and updates weights
        params = update(params, x, y)
    epoch_time = time.time() - start_time

    # Get acc
    train_acc = accuracy(params, train_images, train_labels)
    test_acc = accuracy(params, test_images, test_labels)
    print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
    print("Training set accuracy {}".format(train_acc))
    print("Test set accuracy {}".format(test_acc))


Epoch 0 in 4.15 sec
Training set accuracy 0.9253333210945129
Test set accuracy 0.9266999959945679
Epoch 1 in 3.47 sec
Training set accuracy 0.9427833557128906
Test set accuracy 0.9412999749183655
Epoch 2 in 2.64 sec
Training set accuracy 0.9532666802406311
Test set accuracy 0.9511999487876892
Epoch 3 in 2.85 sec
Training set accuracy 0.9598833322525024
Test set accuracy 0.9557999968528748
Epoch 4 in 3.39 sec
Training set accuracy 0.9651333689689636
Test set accuracy 0.9599999785423279
Epoch 5 in 2.89 sec
Training set accuracy 0.9691500067710876
Test set accuracy 0.9629999995231628
Epoch 6 in 3.70 sec
Training set accuracy 0.9725666642189026
Test set accuracy 0.9648999571800232
Epoch 7 in 3.46 sec
Training set accuracy 0.9754166603088379
Test set accuracy 0.9666999578475952
Epoch 8 in 3.19 sec
Training set accuracy 0.9780833721160889
Test set accuracy 0.9680999517440796
Epoch 9 in 3.53 sec
Training set accuracy 0.9803333282470703
Test set accuracy 0.9691999554634094
